In [1]:
import numpy as np
from itertools import product
from plot.plotly_utils import show_poly_3d, create_poly_3d_gif

## Define an approximation to the earth's shape (oblate spheroid)

Using parameters from the World Geodetic System (WGS 84): https://en.wikipedia.org/wiki/World_Geodetic_System

We exaggerate the oblateness of the earth because the true shape is not obviously different than a sphere in the visualizations. 

In [2]:
OBLATENESS_EXAGGERATION = 1.25

# WGS-84 parameters
A = 6378137 * OBLATENESS_EXAGGERATION  # meters
B = 6356752.3142  # meters

def spheroid(lat, lon):
    '''
    Spheroid surface defined in spherical coordinates (lattitude / longitude).
    Returns points on the spheroid in 3D X/Y/Z coordinates
    '''
    return np.array([
        A*np.cos(lat)*np.cos(lon),
        A*np.cos(lat)*np.sin(lon),
        B*np.sin(lat)
    ])

## Define the mesh nodes

We will combine concepts of a [UV Sphere](https://docs.blender.org/manual/en/latest/modeling/geometry_nodes/mesh_primitives/uv_sphere.html) and the [Geographic Coordinate System](https://en.wikipedia.org/wiki/Geographic_coordinate_system) to construct our spheroid mesh.

The mesh nodes are created by sampling `spheroid()` over a grid of latitude / longitude values.

In [3]:
def uv_sphere_nodes(n_lat=8, n_lon=15):
    nodes = []

    # create the nodes
    latitudes = np.linspace(np.pi/2.0, -np.pi/2.0, num=n_lat, endpoint=True)
    longitudes = np.linspace(-np.pi, np.pi, num=n_lon, endpoint=False)

    # top point
    nodes.append(spheroid(latitudes[0], 0.0))

    # middle points
    for lat, lon in product(latitudes[1:-1], longitudes):
        nodes.append(spheroid(lat, lon))

    # bottom point
    nodes.append(spheroid(latitudes[-1], 0.0))

    return np.array(nodes)


In [4]:
show_poly_3d(nodes=uv_sphere_nodes(n_lat=15, n_lon=60))
#create_poly_3d_gif('./image/uv_spheroid_nodes_dense.gif', nodes=uv_sphere_nodes(n_lat=15, n_lon=60), duration=2)

## Define the polygons

The mesh's polygons are defined by defining connections between groups of three nodes. We will define the UV spheroid from three types of polygon: top, middle, and bottom.

In [5]:
N_LAT = 8
N_LON = 15
nodes = uv_sphere_nodes(n_lat=N_LAT, n_lon=N_LON)

show_poly_3d(nodes=nodes)
#create_poly_3d_gif('./image/uv_spheroid_nodes.gif', nodes=nodes, duration=2)

Define the top polygons by connecting each edge in the first line of latitude with the top node

In [6]:
top = [[0, j + 1, j + 2] for j in range(N_LON-1)]
top.append([0, N_LON, 1])

show_poly_3d(nodes=nodes, polys=top, show_nodes=True)
#create_poly_3d_gif('./image/uv_spheroid_top.gif', nodes=nodes, polys=top, duration=4, show_nodes=True)

Define the middle polygons by subdividing each quadrilateral face between non-terminal parallels

In [7]:
middle = []
for i in range(1, N_LAT-2):
    for j in range(N_LON - 1):
        middle.append([(i-1)*N_LON + j + 1, i*N_LON + j + 1, i*N_LON + j + 2])
        middle.append([(i-1)*N_LON + j + 1, i*N_LON + j + 2, (i-1)*N_LON + j + 2])       
    middle.append([i*N_LON, (i+1)*N_LON, i*N_LON + 1])
    middle.append([i*N_LON, i*N_LON + 1, (i-1)*N_LON + 1]) 

# plot only every other band
alternating_bands = [band for i, band in enumerate(middle) if (i//(2*N_LON))%2]

show_poly_3d(nodes=nodes, polys=alternating_bands, show_nodes=True)
#create_poly_3d_gif('./image/uv_spheroid_middle_bands.gif', nodes=nodes, polys=alternating_bands, duration=4, show_nodes=True)

Define the bottom similarly to the top

In [8]:

bottom = [[(N_LAT-3)*N_LON + j + 1, (N_LAT-2)*N_LON + 1, (N_LAT-3)*N_LON + j + 2] for j in range(N_LON - 1)]
bottom.append([(N_LAT-2)*N_LON, (N_LAT-2)*N_LON + 1, (N_LAT-3)*N_LON + 1])


show_poly_3d(nodes=nodes, polys=bottom, show_nodes=True)
#create_poly_3d_gif('./image/uv_spheroid_bottom.gif', nodes=nodes, polys=bottom, duration=4, show_nodes=True)

### Putting the UV Sphere Together

In [9]:
polys = top + middle + bottom

show_poly_3d(nodes=nodes, polys=polys)
#create_poly_3d_gif('./image/uv_spheroid_full.gif', nodes=nodes, polys=polys, duration=4)